# 1.CoNLL_Parser

In [1]:
path_test_conll_file1 = '../data/conll_test_files/Alliance114418822/Alliance114418822_fold1.conll'
path_test_conll_file2 = '../data/conll_test_files/stanford_ner_without_start_without_empty_space.tsv'

In [2]:
! ls ../data/conll_test_files/

Alliance114418822


In [4]:
# first 10 lines in test file
! head -40 ../data/conll_test_files/Alliance114418822/Alliance114418822_fold1.conll

At    O
the    O
same    O
time    O
,    O
however    O
,    O
the    O
party    O
did    O
not    O
join    O
its    O
Warsaw    B-Alliance114418822
Pact    I-Alliance114418822
brethren    O
in    O
de    O
-    O
Stalinization    O
.    O

On    O
the    O
outside    O
too    O
,    O
the    O
PMR    O
,    O
leading    O
a    O
country    O
that    O
had    O
joined    O
the    O
Warsaw    B-Alliance114418822
Pact    I-Alliance114418822
,    O


In [5]:
import sklearn.preprocessing
# import utils
import collections
import codecs
# import utils_nlp
import re
import time
import token
import os
import pickle
import random

In [20]:
def CoNLL_Parser(filepath):
    """
    Need CoNLL file using -DOCSTART- or empty line for sentence sparator
    """
    count_token = collections.Counter()
    count_label = collections.Counter()
    count_character = collections.Counter()

    tokens = []
    labels = []
    sentence_tokens = []
    sentence_labels = []
    if filepath:
        with open(filepath, 'r') as f:
            for line in f:
                line = line.strip().replace('    ', '\t').split('\t')
                
                # Save previous sentence at the end of it and start to the next sentence
                if len(line) == 0 or len(line[0]) == 0 or '-DOCSTART-' in line[0]:
                    if len(sentence_tokens) > 0:
                        labels.append(sentence_labels)
                        tokens.append(sentence_tokens)
                        sentence_tokens = []
                        sentence_labels = []
                    continue
                
                token = str(line[0])
                label = str(line[-1])
                count_token.update({token: 1})
                count_label.update({label: 1})               
                sentence_tokens.append(token)
                sentence_labels.append(label)
                for character in token:
                    count_character.update({character: 1})

            if len(sentence_tokens) > 0:
                labels.append(sentence_labels)
                tokens.append(sentence_tokens)

    return labels, tokens, count_token, count_label, count_character

In [21]:
labels, tokens, token_count, label_count, character_count = CoNLL_Parser(path_test_conll_file1)

In [22]:
labels[0]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-Alliance114418822',
 'I-Alliance114418822',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [23]:
tokens[0]

['At',
 'the',
 'same',
 'time',
 ',',
 'however',
 ',',
 'the',
 'party',
 'did',
 'not',
 'join',
 'its',
 'Warsaw',
 'Pact',
 'brethren',
 'in',
 'de',
 '-',
 'Stalinization',
 '.']

In [38]:
print('size of chars vocabu :',len(character_count.keys()))
character_count.most_common(100)[-20:]

size of chars vocabu : 127


[('&', 5),
 ('’', 5),
 ('ö', 4),
 ('…', 4),
 ('ū', 4),
 ('è', 4),
 ('ō', 4),
 ('ć', 3),
 ('Ş', 3),
 ('ș', 3),
 ('ı', 3),
 ('в', 2),
 ('ë', 2),
 ('ł', 2),
 ('ā', 2),
 ('[', 2),
 ('ä', 2),
 (']', 2),
 ('ç', 2),
 ('ó', 2)]

### Problem the size of bigger than excepted.

# 2.Token2idx + Deal with UNK

In [ ]:

token_to_index = {}
token_to_index[self.UNK] = self.UNK_TOKEN_INDEX
iteration_number = 0
number_of_unknown_tokens = 0

for token, count in token_count['all'].items():
    if iteration_number == self.UNK_TOKEN_INDEX: iteration_number += 1

    if parameters['remap_unknown_tokens_to_unk'] == 1 and \
        (token_count['train'][token] == 0 or \
        parameters['load_only_pretrained_token_embeddings']) and \
        not utils_nlp.is_token_in_pretrained_embeddings(token, token_to_vector, parameters) and \
        token not in all_tokens_in_pretraining_dataset:
        token_to_index[token] =  self.UNK_TOKEN_INDEX
        number_of_unknown_tokens += 1
        self.tokens_mapped_to_unk.append(token)
    else:
        token_to_index[token] = iteration_number
        iteration_number += 1
